In [1]:
import torch
import torchvision
import torchvision.transforms as transform

In [2]:
import torch
import numpy as np
from transformers import AutoModelForImageClassification, AutoImageProcessor
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
transform = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5] , std=[0.5,0.5,0.5])
])


#model_path='test_food/temp_outputs/'

train_folder_path = 'test_food/data/pizza_steak_sushi/train'
test_folder_path = 'test_food/data/pizza_steak_sushi/test/'


train_dataset = datasets.ImageFolder(root=train_folder_path
                                    ,transform=transform)
test_dataset = datasets.ImageFolder(root=test_folder_path
                                    ,transform=transform)


# Load the CIFAR-10 dataset
#trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


# Create data loaders for batch processing
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)


# Define the class labels
classes = train_dataset.classes



In [4]:
train_dataset.classes

['pizza', 'steak', 'sushi']

In [5]:
import torch.nn as nn
import timm

# Load the Swin Transformer model
model = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=3)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/home/thanawit/anaconda3/envs/twin-pseudo/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343962757/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(50):  # Number of epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data[0].to(device), data[1].to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass, backward pass, and optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Training finished.')

Training finished.


In [7]:
correct = 0
total = 0
model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test set: %.2f %%' % (100 * correct / total))

Accuracy on the test set: 33.33 %


In [20]:
import matplotlib.pyplot as plt
import numpy as np

# Function to show an image
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# Show images and their predicted labels
imshow(torchvision.utils.make_grid(images))
print('Ground truth labels:', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

# Make predictions
images = images.to(device)
outputs = model(images)
_, predicted = torch.max(outputs, 1)

print('Predicted labels:   ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'